In [ ]:
# Libraries
import ollama
import json

In [2]:
## 1 Define Python Tool (Function)
def get_current_weather(location: str, unit: str = "celcius") -> str:
    """
    Gets the current weather for a specified location.
    Args:
        location (str): The city or region to get weather for.
        unit (str): The unit of temperature, either 'celsius' or 'fahrenheit'.
    Returns:
        str: A description of the weather.
    """
    # In a real application you would make an API call
    if location.lower() == "manchester":
        if unit.lower() == "fahrenheit":
            return "Currently 59°F and partly cloudy in Manchester."
        else:
            return "Currently 15°C and partly cloudy in Manchester."
    elif location.lower() == "london":
        return f"Currently 20°C and sunny in London. Unit: {unit}."
    else:
        return f"Sorry, I don't have weather data for {location}."
    

In [3]:
## 2 Describe your tool to the LLM (JSON Schema)
# This is how you tell the LLM about the function's name, purpose, and parameters.
tools_schema = [
    {
        "name": "get_current_weather", # Must match the Python function name exactly
        "description": "Gets the current weather for a specified location.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city or region to get weather for. E.g., 'Manchester', 'New York'."
                },
                "unit": {
                    "type": "string",
                    "description": "The unit of temperature, either 'celsius' or 'fahrenheit'. Defaults to 'celsius'.",
                    "enum": ["celsius", "fahrenheit"] # Optional: provide allowed values
                }
            },
            "required": ["location"] # 'location' is a mandatory parameter
        }
    }
]


In [4]:
# 3 Set up LLM interaction
def chat_with_llm_with_tools(user_query: str, model_name: str = "qwen3:8b"):
    """
    Sends a query to the LLM, including tool definitions, and processes the response.
    """
    
    messages = [
        {
            "role": "system",
            "content": """
            You are a helpful AI assistant with access to tools.
            You should use the 'get_current_weather' tool to answer questions about the weather.
            Always prefer calling a tool if the user's intent matches a tool's function.
            """
        },
        {
            "role": "user",
            "content": user_query
        }
    ]
    
    print(f"\n--- User Query: '{user_query}' ---") # print out the user query
    print(f"--- Sending to Ollama model: '{model_name}' with defined tools ---")
    
    try:
        # give input to the model
        response = ollama.chat( 
            model=model_name,
            messages=messages,
            tools=tools_schema,
            options={
                "temperature": 0.0 
            }
        )
        
        # 4. Process the LLMs Response
        # Check 
        if 'tool_calls' in response['message'] and response['message']['tool_calls']:
            print("\n--- LLM decided to call a tool! ---")
            for tool_call in response['message']['tool_calls']:
                tool_name = tool_call['function']['name']
                tool_args = tool_call['function']['arguments']
                tool_id = tool_call['id'] # Each tool call has a unique ID

                print(f"  Tool Name: {tool_name}")
                print(f"  Tool Arguments: {json.dumps(tool_args, indent=2)}")

                # Here, you would execute the actual Python function based on tool_name
                if tool_name == "get_current_weather":
                    # Call your Python function with the arguments extracted by the LLM
                    weather_result = get_current_weather(**tool_args)
                    print(f"\n--- Executing tool '{tool_name}'... ---")
                    print(f"  Tool Execution Result: {weather_result}")
                    
        else:
            # LLM returned a direct text response (no tool call)
            print("\n--- LLM returned a direct text response (no tool call): ---")
            print(response['message']['content'])
        
        
    except Exception as e:
        print(f"An error occurred {e}")

    return response



In [5]:
# Run examples
r = chat_with_llm_with_tools(user_query = "What's the weather like in Manchester?")
t = chat_with_llm_with_tools(user_query = "Tell me a fun fact about giraffes.")


--- User Query: 'What's the weather like in Manchester?' ---
--- Sending to Ollama model: 'qwen3:8b' with defined tools ---

--- LLM decided to call a tool! ---
An error occurred 'id'

--- User Query: 'Tell me a fun fact about giraffes.' ---
--- Sending to Ollama model: 'qwen3:8b' with defined tools ---

--- LLM returned a direct text response (no tool call): ---
<think>
Okay, the user asked for a fun fact about giraffes. Let me think. I need to check if any of the provided tools can help with that. The tools section is empty, so there's no function to call. Since the user's request is just for information and not related to weather or any other tool functions, I should answer directly. I'll share a fun fact about giraffes without using any tools.
</think>

Here's a fun fact about giraffes: Did you know that a giraffe's neck is actually about the same length as an elephant's neck? The misconception that giraffes have long necks to reach leaves high up in trees is a classic example of 

In [ ]:
r['message']['tool_calls'], t['message']

In [25]:
import ollama
import json

# --- 1. Define your Python Tool (Function) ---
# This is the actual function that performs the desired action.
def get_current_weather(location: str, unit: str = "celsius") -> str:
    """
    Gets the current weather for a specified location.
    Args:
        location (str): The city or region to get weather for.
        unit (str): The unit of temperature, either 'celsius' or 'fahrenheit'.
    Returns:
        str: A description of the weather.
    """
    # In a real application, you would make an API call here (e.g., to OpenWeatherMap)
    # For this basic example, we'll return a hardcoded response.
    if location.lower() == "manchester":
        if unit.lower() == "fahrenheit":
            return "Currently 59°F and partly cloudy in Manchester."
        else:
            return "Currently 15°C and partly cloudy in Manchester."
    elif location.lower() == "london":
        return f"Currently 20°C and sunny in London. Unit: {unit}."
    else:
        return f"Sorry, I don't have weather data for {location}."

# --- 2. Describe your Tool to the LLM (JSON Schema) ---
# This is how you tell the LLM about the function's name, purpose, and parameters.
tools_schema = [
    {
        "name": "get_current_weather", # Must match the Python function name exactly
        "description": "Gets the current weather for a specified location.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city or region to get weather for. E.g., 'Manchester', 'New York'."
                },
                "unit": {
                    "type": "string",
                    "description": "The unit of temperature, either 'celsius' or 'fahrenheit'. Defaults to 'celsius'.",
                    "enum": ["celsius", "fahrenheit"] # Optional: provide allowed values
                }
            },
            "required": ["location"] # 'location' is a mandatory parameter
        }
    }
]

# --- 3. Set up the LLM Interaction ---
def chat_with_llm_with_tools(user_query: str, model_name: str = "qwen3:8b"):
    """
    Sends a query to the LLM, including tool definitions, and processes the response.
    """
    messages = [
        {
            "role": "system",
            "content": """
            You are a helpful AI assistant with access to tools.
            You should use the 'get_current_weather' tool to answer questions about the weather.
            Always prefer calling a tool if the user's intent matches a tool's function.
            """
        },
        {
            "role": "user",
            "content": user_query
        }
    ]

    print(f"\n--- User Query: '{user_query}' ---")
    print(f"--- Sending to Ollama model: '{model_name}' with defined tools ---")

    try:
        response = ollama.chat(
            model=model_name,
            messages=messages,
            tools=tools_schema, # This is where you pass the tool definitions
            options={
                "temperature": 0.0 # Make it more deterministic for tool calling demos
            }
        )

        # --- 4. Process the LLM's Response ---
        # Check if the LLM decided to call a tool
        if 'tool_calls' in response['message'] and response['message']['tool_calls']:
            print("\n--- LLM decided to call a tool! ---")
            for tool_call in response['message']['tool_calls']:
                tool_name = tool_call['function']['name']
                tool_args = tool_call['function']['arguments']
                #tool_id = tool_call['id'] # Each tool call has a unique ID

                print(f"  Tool Name: {tool_name}")
                print(f"  Tool Arguments: {json.dumps(tool_args, indent=2)}")

                # Here, you would execute the actual Python function based on tool_name
                if tool_name == "get_current_weather":
                    # Call your Python function with the arguments extracted by the LLM
                    weather_result = get_current_weather(**tool_args)
                    print(f"\n--- Executing tool '{tool_name}'... ---")
                    print(f"  Tool Execution Result: {weather_result}")

                    # Optionally: Send the tool's output back to the LLM for it to summarize or refine
                    # This is how multi-turn reasoning with tools works.
                    # messages.append({"role": "tool", "content": weather_result, "tool_call_id": tool_id})
                    # final_response = ollama.chat(model=model_name, messages=messages)
                    # print(f"\n--- LLM's final response after tool execution: ---")
                    # print(final_response['message']['content'])

                else:
                    print(f"Error: Unknown tool '{tool_name}' requested by LLM.")
        else:
            # LLM returned a direct text response (no tool call)
            print("\n--- LLM returned a direct text response (no tool call): ---")
            print(response['message']['content'])

    except Exception as e:
        print(f"An error occurred: {e}")
        print("Please ensure your Ollama server is running and the model is downloaded (`ollama run llama3.2`).")

# --- Run the Example Queries ---
if __name__ == "__main__":
    # Test case 1: User asks a question that requires the tool
    #chat_with_llm_with_tools("What's the weather like in Manchester?", model_name="qwen3:8b")

    # Test case 2: User asks with a specific unit
    chat_with_llm_with_tools("How many degrees Fahrenheit is it in London?", model_name="qwen3:8b")

    # Test case 3: User asks about a location not handled by our dummy tool
    #chat_with_llm_with_tools("What's the temperature in Paris?", model_name="qwen3:8b")

    # Test case 4: User asks a general question (should not call tool)
    #chat_with_llm_with_tools("Tell me a fun fact about giraffes.", model_name="qwen3:8b")


--- User Query: 'How many degrees Fahrenheit is it in London?' ---
--- Sending to Ollama model: 'qwen3:8b' with defined tools ---

--- LLM decided to call a tool! ---
  Tool Name: get_current_weather
  Tool Arguments: {
  "location": "London",
  "unit": "Fahrenheit"
}

--- Executing tool 'get_current_weather'... ---
  Tool Execution Result: Currently 20°C and sunny in London. Unit: Fahrenheit.


In [ ]:
## Very simple tool calling
def add_two_numbers(a: int, b: int) -> int:
  """
  Add two numbers

  Args:
    a: The first integer number
    b: The second integer number

  Returns:
    int: The sum of the two numbers
  """
  return a + b

def subtract_two_numbers(a: int, b: int) -> int:
  """
  Add two numbers

  Args:
    a: The first integer number
    b: The second integer number

  Returns:
    int: Subtraction of one number from another
  """
  return a - b


response = ollama.chat(
  'qwen3:8b',
  messages=[{'role': 'user', 'content': 'What is 10 - 10?'}],
  tools=[add_two_numbers, subtract_two_numbers], # Actual function reference
)


In [ ]:
response_sub = ollama.chat(
  'qwen3:8b',
  messages=[{'role': 'user', 'content': 'Select a suitable tool for the problem. If no suitable tool available then tell a joke'}, {'role': 'user', 'content': 'What is 10 - 10?'}],
  tools=[add_two_numbers, subtract_two_numbers], # Actual function reference
)

In [20]:
response_joke = ollama.chat(
  'qwen3:8b',
  messages=[{'role': 'user', 'content': 'Select a suitable tool for the problem. If no suitable tool available then tell a joke'}, {'role': 'user', 'content': 'What is the captial of India?'}],
  tools=[add_two_numbers, subtract_two_numbers], # Actual function reference
)

In [24]:
response.message.tool_calls, response_sub.message.tool_calls
#print(response_joke.message)

([ToolCall(function=Function(name='add_two_numbers', arguments={'a': 10, 'b': 10}))],
 [ToolCall(function=Function(name='subtract_two_numbers', arguments={'a': 10, 'b': 10}))])